# VacationPy

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
#creating path to output data csv
weatherInfoFile = "../WeatherPy/output_data/cities.csv"

#reading in csv
uglyWeather = pd.read_csv(weatherInfoFile)

#deletes the first column named unnamed, which counted from 0-the end of the datset for no reason. 
weather = uglyWeather.drop(columns=["Unnamed: 0"])
weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat.,Lng.,Max Temp,Wind Speed
0,Cape Town,75,ZA,1579969097,82,-33.93,18.42,70.00,14.99
1,Hong Kong,100,HK,1579969300,93,22.29,114.16,64.99,8.86
2,Sitka,100,US,1579969383,81,57.05,-135.33,31.93,6.33
3,Bredasdorp,4,ZA,1579969356,54,-34.53,20.04,78.80,14.99
4,Avera,40,US,1579969383,65,33.19,-82.53,50.00,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#Creating coordinates and humidity lists which will be added to the heatmap
coordinates = weather[['Lat.','Lng.']]
humidity = weather['Humidity']

#Map center will focus the figure below on Kriti, in the mediterranean
#sea to give the map a nice focal point
mapCenter = (35.2401, 24.8093)

# Plot Heatmap, setting center of map, and zoom level to 2 to get rid of 
#multiple maps in same fig
fig = gmaps.figure(center=mapCenter,zoom_level=2)

#heat map with weight set to humidity 
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                                    max_intensity=100,point_radius = 5)

#adding the heatmap layer to the figure
fig.add_layer(heatmap_layer)

#display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#Drop country, date,lat, and long values to focus on weather data and city names 
updated_df = pd.DataFrame(weather, columns = ["City","Cloudiness","Max Temp", "Wind Speed", "Humidity"])

#Now I get to be picky and set the values of the weather that I enjoy most
#Temperature should be 65-80 F
#Wind should be minimal, no more than 10 mph
#Clear sky is better than cloudy sky, I set the max cloudiness value to 20%
#I dislike humidity so I set te value for humidity at less than 50%
#Drop all rows that do not fit this criteria

idealCloud = updated_df["Cloudiness"] <=20
idealTemp = ((updated_df["Max Temp"] <=80) & (updated_df["Max Temp"] > 65))
idealWind = updated_df["Wind Speed"] < 10
idealHumidity = updated_df["Humidity"] < 50

#narrowing table down to show preferred conditions, creating new dataframe from that table
idealCities = updated_df[idealCloud & idealTemp & idealWind & idealHumidity]
narrowed_city_df = pd.DataFrame(idealCities)
narrowed_city_df 

,City,Cloudiness,Max Temp,Wind Speed,Humidity
7,Phalodi,1,67.87,2.55,33
23,Clearwater,1,66.99,9.17,39
43,Broken Hill,0,71.01,5.57,28
55,Alice Springs,0,77.02,8.39,22
337,Doha,0,66.00,5.01,37
487,Kot Diji,0,67.87,1.57,38
498,Bayonet Point,1,66.00,9.17,39


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#creating a dataframe which is strictly geographical with no weather information
hotel_df = pd.DataFrame(weather, columns = ["City" , "Country", "Lat.", "Lng."])

#creating empty column to hold hotel names later and merging with ideal vacation spots
hotel_df["Hotel Name"] = ''
hotel_df = hotel_df.merge(narrowed_city_df)
hotel_df

,City,Country,Lat.,Lng.,Hotel Name,Cloudiness,Max Temp,Wind Speed,Humidity
0,Phalodi,IN,27.13,72.37,,1,67.87,2.55,33
1,Clearwater,US,27.97,-82.80,,1,66.99,9.17,39
2,Broken Hill,AU,-31.95,141.43,,0,71.01,5.57,28
3,Alice Springs,AU,-23.70,133.88,,0,77.02,8.39,22
4,Doha,QA,25.29,51.53,,0,66.00,5.01,37
5,Kot Diji,PK,27.34,68.71,,0,67.87,1.57,38
6,Bayonet Point,US,28.33,-82.68,,1,66.00,9.17,39


# Calling the Google API

In [15]:
#empty list to hold hotel info later
hotels = []

#counter variable
counter = 0

#for loop which will loop through cities in the ideal weather cities 
for cities in hotel_df['City']:
    #the target variables set what the parameters will be, as the loop is iterated
    target_coordinates = str(hotel_df['Lat.'][counter])+' , '+str(hotel_df['Lng.'][counter])
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"
    counter +=1
    #parameters get passed to the url when calling the api to define the search
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}
    
    #base url for the api which we will be calling
    test_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #try except statement with requests embedded, this will prevent errors when trying to fill the hotel name list
    #due to missing information
    try:  
        response = requests.get(test_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)
        
#appending dataframe with hotel names
hotel_df['Hotel Name'] = hotels

In [16]:
#cleaning up the dataframe
clean_hotels = hotel_df.dropna()

#removing columns related to weather information
clean_hotels = clean_hotels.drop(columns = ["Cloudiness", "Max Temp", 'Wind Speed','Humidity'])

#final clean hotels dataframe for my perfect vacation
clean_hotels

,City,Country,Lat.,Lng.,Hotel Name
0,Phalodi,IN,27.13,72.37,Kurja Resort
1,Clearwater,US,27.97,-82.80,Hilton Clearwater Beach Resort & Spa
2,Broken Hill,AU,-31.95,141.43,Palace Hotel
3,Alice Springs,AU,-23.70,133.88,DoubleTree by Hilton Hotel Alice Springs
4,Doha,QA,25.29,51.53,Warwick Doha Hotel
6,Bayonet Point,US,28.33,-82.68,Homewood Suites by Hilton Tampa-Port Richey


In [17]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotels.iterrows()]
locations = clean_hotels[["Lat.", "Lng."]]

In [18]:
#Map center will focus the figure below on Kriti, in the mediterranean
#sea to give the map a nice focal point
mapCenter = (35.2401, 24.8093)

# Plot Heatmap, setting center of map, and zoom level to 2 to get rid of 
# multiple maps in same fig
fig = gmaps.figure(center=mapCenter,zoom_level=2.0)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))